<a href="https://colab.research.google.com/github/nntu079/PosTagging/blob/main/Postagging%20BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import necessary libraries
import warnings
import keras
warnings.filterwarnings("ignore")

import numpy as np

from matplotlib import pyplot as plt

from nltk.corpus import brown
from nltk.corpus import treebank
from nltk.corpus import conll2000

import seaborn as sns

from gensim.models import KeyedVectors

from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Input
from keras.layers import TimeDistributed
from keras.layers import LSTM, GRU, Bidirectional, SimpleRNN, RNN
from keras.models import Model
from keras.preprocessing.text import Tokenizer

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import nltk
nltk.download('treebank')
nltk.download('brown')
nltk.download('conll2000')
nltk.download('universal_tagset')

import pickle

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [ ]:
# load POS tagged corpora from NLTK
treebank_corpus = treebank.tagged_sents(tagset='universal')
brown_corpus = brown.tagged_sents(tagset='universal')
conll_corpus = conll2000.tagged_sents(tagset='universal')
tagged_sentences = treebank_corpus + brown_corpus + conll_corpus

In [ ]:
X = [] # store input sequence
Y = [] # store output sequence

for sentence in tagged_sentences:
    X_sentence = []
    Y_sentence = []
    for entity in sentence:         
        X_sentence.append(entity[0])  # entity[0] contains the word
        Y_sentence.append(entity[1])  # entity[1] contains corresponding tag
        
    X.append(X_sentence)
    Y.append(Y_sentence)
print(X[0])
print(Y[0])


['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.']
['NOUN', 'NOUN', '.', 'NUM', 'NOUN', 'ADJ', '.', 'VERB', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'NOUN', 'NUM', '.']


In [ ]:
X.append(['I','can','can','a','can','.'])
Y.append(['PRON','VERB','VERB','DET','NOUN','.'])

X.append(['I','can','can','a','can','.'])
Y.append(['PRON','VERB','VERB','DET','NOUN','.'])

X.append(['I','can','can','a','can','.'])
Y.append(['PRON','VERB','VERB','DET','NOUN','.'])

X.append(['I','can','can','a','can','.'])
Y.append(['PRON','VERB','VERB','DET','NOUN','.'])

X.append(['I','can','can','a','can','.'])
Y.append(['PRON','VERB','VERB','DET','NOUN','.'])

print( set((Y[-1]+Y[0]+Y[1]+Y[2]+Y[3]+Y[4]+Y[5]+Y[6]+Y[7]+Y[9]+Y[8]+Y[10]+Y[11]+Y[12]+Y[13]+Y[14]+Y[15]+Y[16])))

{'CONJ', 'X', 'ADJ', 'VERB', 'DET', 'ADP', 'ADV', 'NUM', 'NOUN', 'PRON', 'PRT', '.'}


In [ ]:
# encode X
word_tokenizer = Tokenizer()                      # instantiate tokeniser
word_tokenizer.fit_on_texts(X)                    # fit tokeniser on data
X_encoded = word_tokenizer.texts_to_sequences(X)  # use the tokeniser to encode input sequence

with open('word_tokenizer.pickle', 'wb') as handle:
    pickle.dump(word_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


# encode Y
tag_tokenizer = Tokenizer()
tag_tokenizer.fit_on_texts(Y)
Y_encoded = tag_tokenizer.texts_to_sequences(Y)

with open('tag_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tag_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

print(X_encoded[0])
print(Y_encoded[0])

[6423, 24231, 2, 7652, 102, 170, 2, 47, 1898, 1, 269, 17, 7, 13230, 619, 1711, 2761, 3]
[1, 1, 3, 11, 1, 6, 3, 2, 2, 5, 1, 4, 5, 6, 1, 1, 11, 3]


In [ ]:
# look at first encoded data point

print("** Raw data point **", "\n", "-"*100, "\n")
print('X: ', X[0], '\n')
print('Y: ', Y[0], '\n')
print()
print("** Encoded data point **", "\n", "-"*100, "\n")
print('X: ', X_encoded[0], '\n')
print('Y: ', Y_encoded[0], '\n')

** Raw data point ** 
 ---------------------------------------------------------------------------------------------------- 

X:  ['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.'] 

Y:  ['NOUN', 'NOUN', '.', 'NUM', 'NOUN', 'ADJ', '.', 'VERB', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'NOUN', 'NUM', '.'] 


** Encoded data point ** 
 ---------------------------------------------------------------------------------------------------- 

X:  [6423, 24231, 2, 7652, 102, 170, 2, 47, 1898, 1, 269, 17, 7, 13230, 619, 1711, 2761, 3] 

Y:  [1, 1, 3, 11, 1, 6, 3, 2, 2, 5, 1, 4, 5, 6, 1, 1, 11, 3] 



In [ ]:
# sequences greater than 100 in length will be truncated
MAX_SEQ_LENGTH = 100
X_padded = pad_sequences(X_encoded, maxlen=MAX_SEQ_LENGTH, padding="pre", truncating="post")
Y_padded = pad_sequences(Y_encoded, maxlen=MAX_SEQ_LENGTH, padding="pre", truncating="post")# print the first sequence
print(X_padded[0], "\n"*3)
print(Y_padded[0])

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0  6423 24231
     2  7652   102   170     2    47  1898     1   269    17     7 13230
   619  1711  2761     3] 



[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  1  1  3 11  1  6  3  2  2  5  1  4  5  6
  1  1 11  3]


In [ ]:
X, Y = X_padded, Y_padded

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/My Drive/Kaggle/GoogleNews-vectors-negative300.bin' #Tú chạy thì sửa lại đường dẫn phù hợp nha
#đường dẫn của Tú: /content/drive/My Drive/Môn học/Kaggle/GoogleNews-vectors-negative300.bin
path='/content/drive/My Drive/Môn học/Kaggle/GoogleNews-vectors-negative300.bin'

word2vec = KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
EMBEDDING_SIZE  = 300  # each word in word2vec model is represented using a 300 dimensional vector
VOCABULARY_SIZE = len(word_tokenizer.word_index) + 1

# create an empty embedding matix
embedding_weights = np.zeros((VOCABULARY_SIZE, EMBEDDING_SIZE))

# create a word to index dictionary mapping
word2id = word_tokenizer.word_index

# copy vectors from word2vec model to the words present in corpus
for word, index in word2id.items():    
    try:
        embedding_weights[index, :] = word2vec[word]
    except KeyError:
        pass
        
Y = to_categorical(Y)

In [ ]:
# split entire data into training and testing sets
TEST_SIZE = 0.05
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=TEST_SIZE, random_state=4)

VALID_SIZE = 0.25
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size=VALID_SIZE, random_state=4)

# print number of samples in each set
print("TRAINING DATA")
print('Shape of input sequences: {}'.format(X_train.shape))
print('Shape of output sequences: {}'.format(Y_train.shape))
print("-"*50)
print("VALIDATION DATA")
print('Shape of input sequences: {}'.format(X_validation.shape))
print('Shape of output sequences: {}'.format(Y_validation.shape))
print("-"*50)
print("TESTING DATA")
print('Shape of input sequences: {}'.format(X_test.shape))
print('Shape of output sequences: {}'.format(Y_test.shape))

TRAINING DATA
Shape of input sequences: (51522, 100)
Shape of output sequences: (51522, 100, 14)
--------------------------------------------------
VALIDATION DATA
Shape of input sequences: (17174, 100)
Shape of output sequences: (17174, 100, 14)
--------------------------------------------------
TESTING DATA
Shape of input sequences: (3616, 100)
Shape of output sequences: (3616, 100, 14)


In [ ]:
NUM_CLASSES = Y.shape[2]

In [ ]:
# create architecture

rnn_model = Sequential()

# create embedding layer - usually the first layer in text problems
rnn_model.add(Embedding(input_dim     =  VOCABULARY_SIZE,         # vocabulary size - number of unique words in data
                        output_dim    =  EMBEDDING_SIZE,          # length of vector with which each word is represented
                        input_length  =  MAX_SEQ_LENGTH,          # length of input sequence
                        weights       = [embedding_weights],      # word embedding matrix
                        trainable     =  True                     # True - update the embeddings while training
))

# add an RNN layer which contains 64 RNN cells
rnn_model.add(Bidirectional(SimpleRNN(64, 
              return_sequences=True  # True - return whole sequence; False - return single output of the end of the sequence
)))

# add time distributed (output at each sequence) layer
rnn_model.add(TimeDistributed(Dense(NUM_CLASSES, activation='softmax')))

In [ ]:
rnn_model.compile(loss      =  'categorical_crossentropy',
                  optimizer =  'adam',
                  metrics   =  ['acc'])

In [ ]:
# check summary of the model
rnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 300)          17834700  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100, 128)          46720     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 100, 14)           1806      
Total params: 17,883,226
Trainable params: 17,883,226
Non-trainable params: 0
_________________________________________________________________


In [ ]:
rnn_training = rnn_model.fit(X_train, Y_train, batch_size=128, epochs=10, validation_data=(X_validation, Y_validation))


Epoch 1/10
403/403 [==============================] - 163s 404ms/step - loss: 0.2191 - acc: 0.9503 - val_loss: 0.0480 - val_acc: 0.9877
Epoch 2/10
403/403 [==============================] - 162s 403ms/step - loss: 0.0305 - acc: 0.9914 - val_loss: 0.0279 - val_acc: 0.9911
Epoch 3/10
403/403 [==============================] - 155s 386ms/step - loss: 0.0182 - acc: 0.9942 - val_loss: 0.0242 - val_acc: 0.9920
Epoch 4/10
403/403 [==============================] - 158s 392ms/step - loss: 0.0138 - acc: 0.9955 - val_loss: 0.0228 - val_acc: 0.9924
Epoch 5/10
403/403 [==============================] - 158s 391ms/step - loss: 0.0110 - acc: 0.9965 - val_loss: 0.0222 - val_acc: 0.9925
Epoch 6/10
403/403 [==============================] - 160s 397ms/step - loss: 0.0088 - acc: 0.9973 - val_loss: 0.0222 - val_acc: 0.9926
Epoch 7/10
403/403 [==============================] - 159s 395ms/step - loss: 0.0070 - acc: 0.9980 - val_loss: 0.0227 - val_acc: 0.9926
Epoch 8/10
403/403 [============================

In [ ]:
rnn_model.save('/content/drive/MyDrive/Môn học/NLP/Model/model.h5')

In [ ]:
#Because practice makes perfect, we suggest you start learning how to wrap a gift right now
sample=input()
X_sample=sample.split()
X_sample
X_temp=[]
X_temp.append(X_sample)

In [ ]:
X_temp

[['I', 'can', 'can', 'a', 'can']]

In [ ]:
# encode X
X_sample_encoded = word_tokenizer.texts_to_sequences(X_temp)  # use the tokeniser to encode input sequence
print(X_sample_encoded)

[[29, 69, 69, 7, 69]]


In [ ]:
# sequences greater than 100 in length will be truncated
X_sample_padded = pad_sequences(X_sample_encoded, maxlen=MAX_SEQ_LENGTH, padding="pre", truncating="post")
print(X_sample_padded)

In [ ]:
t=rnn_model.predict_classes(X_sample_padded)

In [ ]:
print(sample)
print(tag_tokenizer.sequences_to_texts(t)[0])
